In [ ]:
!pip install transformers datasets
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import Dataset, DatasetDict
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd

## Load roberta model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli')
roberta_model = AutoModel.from_pretrained('ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/703 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

## Define the CNN model

In [ ]:
class CNNKeywordModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_filters, filter_sizes, output_dim):
        super(CNNKeywordModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([
            nn.Conv2d(1, num_filters, (fs, embedding_dim)) for fs in filter_sizes
        ])
        self.output_dim = output_dim
        self.fc = nn.Linear(num_filters * len(filter_sizes), output_dim)

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)
        conv_results = [F.relu(conv(x)).squeeze(3) for conv in self.convs]
        pooled_results = [F.max_pool1d(cr, cr.size(2)).squeeze(2) for cr in conv_results]
        cat = torch.cat(pooled_results, 1)
        return self.fc(cat)

## Combined Model

In [ ]:
class CombinedModel(nn.Module):
    def __init__(self, roberta_model, cnn_model, hidden_dim, num_classes):
        super(CombinedModel, self).__init__()
        self.roberta_model = roberta_model
        self.cnn_model = cnn_model
        self.fc = nn.Linear(cnn_model.output_dim + roberta_model.config.hidden_size, num_classes)
        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, input_ids=None, attention_mask=None, cnn_input=None, labels=None):
        roberta_outputs = self.roberta_model(input_ids=input_ids, attention_mask=attention_mask)
        roberta_hidden_states = roberta_outputs.last_hidden_state
        cnn_outputs = self.cnn_model(cnn_input)

        combined = torch.cat((roberta_hidden_states[:, 0, :], cnn_outputs), dim=1)
        logits = self.fc(combined)

        if labels is not None:
            loss = self.loss_fn(logits, labels)
            return loss, logits
        else:
            return logits

## Load dataset

### Load data from Huggingface

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict

label_mapping = {'Entailed': 0, 'Neutral': 1, 'Contradict': 2}

def get_train_nli_data(legal_type: str) -> pd.DataFrame:
    justice_lens_dataset = load_dataset("darrow-ai/LegalLensNLI")

    train_df = justice_lens_dataset["train"].filter(lambda example: example["legal_act"] != legal_type).to_pandas()
    train_df['label'] = train_df['label'].map(label_mapping).astype(int)

    return train_df

def get_test_nli_data(legal_type: str) -> pd.DataFrame:
    justice_lens_dataset = load_dataset("darrow-ai/LegalLensNLI")

    test_df = justice_lens_dataset["train"].filter(lambda example: example["legal_act"] == legal_type).to_pandas()
    test_df['label'] = test_df['label'].map(label_mapping).astype(int)

    return test_df

legal_type = "wage"
train_df = get_train_nli_data(legal_type)
test_df = get_test_nli_data(legal_type)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
datasets = DatasetDict({'train': train_dataset, 'validation': test_dataset})


Generating train split:   0%|          | 0/312 [00:00<?, ? examples/s]

Filter:   0%|          | 0/312 [00:00<?, ? examples/s]

Filter:   0%|          | 0/312 [00:00<?, ? examples/s]

### Load data locally

In [ ]:
def tokenize_for_sequence_model(examples):
    return tokenizer(examples['premise'], examples['hypothesis'], truncation=True, padding=True)

def tokenize_for_cnn(examples):
    combined_texts = [p + " " + h for p, h in zip(examples['premise'], examples['hypothesis'])]
    encoding = tokenizer(combined_texts, padding=True, truncation=True, return_tensors="pt")
    return {
        'input_ids': encoding['input_ids'],
        'attention_mask': encoding['attention_mask']
    }


tokenized_datasets = datasets.map(tokenize_for_sequence_model, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['premise', 'hypothesis'])
tokenized_datasets.set_format('torch')

cnn_tokenized_datasets = datasets.map(tokenize_for_cnn, batched=True)
cnn_tokenized_datasets.set_format('torch')

sequence_inputs = tokenized_datasets["train"]
cnn_inputs = cnn_tokenized_datasets["train"]
cnn_input_lists = [tensor.tolist() for tensor in cnn_inputs['input_ids']]
combined_dataset_train = sequence_inputs.add_column('cnn_input', cnn_input_lists)

sequence_inputs_val = tokenized_datasets["validation"]
cnn_inputs_val = cnn_tokenized_datasets["validation"]
cnn_input_lists_val = [tensor.tolist() for tensor in cnn_inputs_val['input_ids']]
combined_dataset_val = sequence_inputs_val.add_column('cnn_input', cnn_input_lists_val)

Map:   0%|          | 0/299 [00:00<?, ? examples/s]

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

Map:   0%|          | 0/299 [00:00<?, ? examples/s]

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

## Train

In [ ]:
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "macro_f1": f1,
        "macro_precision": precision,
        "macro_recall": recall
    }

In [ ]:
vocab_size = tokenizer.vocab_size
embedding_dim = 100
num_filters = 100
filter_sizes = [2, 3, 4]
output_dim = 50

cnn_model = CNNKeywordModel(vocab_size, embedding_dim, num_filters, filter_sizes, output_dim)
combined_model = CombinedModel(roberta_model=roberta_model, cnn_model=cnn_model, hidden_dim=256, num_classes=3)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model='macro_f1',
    greater_is_better=True,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=20,
    weight_decay=0.01,
    save_total_limit=2,
    warmup_steps= 50
)

trainer = Trainer(
    model=combined_model,
    args=training_args,
    train_dataset=combined_dataset_train,
    eval_dataset=combined_dataset_val,
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1,Macro Precision,Macro Recall
1,No log,0.777663,0.615385,0.452381,0.383333,0.555556
2,No log,1.306032,0.769231,0.719048,0.888889,0.694444
3,No log,1.240154,0.846154,0.805556,0.866667,0.833333
4,No log,0.728826,0.846154,0.815629,0.841270,0.805556
5,No log,0.761460,0.923077,0.904762,0.916667,0.916667
6,No log,1.155567,0.846154,0.838095,0.916667,0.805556
7,0.288300,1.203833,0.846154,0.815629,0.841270,0.805556
8,0.288300,0.999322,0.923077,0.904762,0.916667,0.916667
9,0.288300,1.638051,0.846154,0.838095,0.916667,0.805556
10,0.288300,1.584453,0.846154,0.838095,0.916667,0.805556


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=1500, training_loss=0.09745880721819898, metrics={'train_runtime': 4175.6765, 'train_samples_per_second': 1.432, 'train_steps_per_second': 0.359, 'total_flos': 0.0, 'train_loss': 0.09745880721819898, 'epoch': 20.0})

# Save model

In [ ]:
!pip install transformers huggingface_hub

In [ ]:
from huggingface_hub import login

login(token="<Your-write-token>")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import os
import torch

# Define your model directory and Hugging Face repo names
hf_repo_name = "nimamegh/roberta_cnn_legal"
save_directory = './saved_model'
os.makedirs(save_directory, exist_ok=True)

# Save the RoBERTa model and tokenizer locally
roberta_model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

# Push the RoBERTa model and tokenizer to Hugging Face Hub
roberta_model.push_to_hub(hf_repo_name)
tokenizer.push_to_hub(hf_repo_name)

# Save the CNN model's state dict locally
torch.save(cnn_model.state_dict(), os.path.join(save_directory, 'cnn_model.pth'))

# Save the combined model's state dict locally
torch.save(combined_model.state_dict(), os.path.join(save_directory, 'combined_model.pth'))

# Save the training arguments locally
torch.save(training_args, os.path.join(save_directory, 'training_args.bin'))

# Optionally, push other artifacts to Hugging Face Hub
from huggingface_hub import HfApi

# Create a new repository or use an existing one
api = HfApi()
api.upload_file(
    path_or_fileobj=os.path.join(save_directory, 'cnn_model.pth'),
    path_in_repo='cnn_model.pth',
    repo_id=hf_repo_name,
)

api.upload_file(
    path_or_fileobj=os.path.join(save_directory, 'combined_model.pth'),
    path_in_repo='combined_model.pth',
    repo_id=hf_repo_name,
)

api.upload_file(
    path_or_fileobj=os.path.join(save_directory, 'training_args.bin'),
    path_in_repo='training_args.bin',
    repo_id=hf_repo_name,
)



README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

cnn_model.pth:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

combined_model.pth:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/nimamegh/roberta_cnn_legal/commit/85fb5df32dd41523e8ac4ab81c5f7709e6f5da0d', commit_message='Upload training_args.bin with huggingface_hub', commit_description='', oid='85fb5df32dd41523e8ac4ab81c5f7709e6f5da0d', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
!zip -r nli_model.zip /content/saved_model